# 部署与维护

In [1]:
import os
import warnings
from IPython.display import Markdown
from setenv import APIKeyManager

# 忽略警告
warnings.filterwarnings('ignore')
# 设置API密钥
key_manager = APIKeyManager().setup_api_key(["DEEPSEEK_API_KEY"])

# 查看Python版本
!python -V
# 查看安装的库
!pip list | grep -E 'langchain|openai|llm|tiktoken|chromadb|cryptography|duck|unstructured|numpy|scipy'

Python 3.9.21
chromadb                                 0.6.3
cryptography                             44.0.2
duckduckgo_search                        7.5.3
langchain                                0.3.21
langchain-community                      0.3.20
langchain-core                           0.3.47
langchain-deepseek                       0.1.3
langchain-experimental                   0.3.4
langchain-openai                         0.3.9
langchain-text-splitters                 0.3.7
numpy                                    2.0.2
openai                                   1.68.2
scipy                                    1.10.1
tiktoken                                 0.9.0
unstructured                             0.17.2
unstructured-client                      0.31.3


## 1. 应用部署

### 1.1 API服务

In [2]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from langchain_deepseek import ChatDeepSeek
from langchain.prompts import PromptTemplate
import os
import logging

# 配置日志
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

app = FastAPI()

class Query(BaseModel):
    text: str

@app.post("/generate")
async def generate(query: Query):
    try:
        logger.info(f"收到请求: {query.text}")
        llm = ChatDeepSeek(model='deepseek-chat')
        prompt = PromptTemplate(
            template=
            """
            结果使用中英两种语言回答
            问题: {question}
            """,
            input_variables=['question']
        )
        chain = prompt | llm
        response = chain.invoke(query.text)
        logger.info("请求处理成功")
        return {"response": response}
    except Exception as e:
        error_msg = f"处理请求时发生错误: {str(e)}"
        logger.error(error_msg)
        raise HTTPException(status_code=500, detail=error_msg)

In [3]:
import requests
import json
import time
import uvicorn
import asyncio
import uvloop
from threading import Thread

# 在后台线程中启动服务
def start_server():  
    # Use standard asyncio event loop instead of uvloop  
    uvicorn.run(app, host="0.0.0.0", port=8000, loop="asyncio")  

# 启动服务器 
server_thread = Thread(target=start_server)
server_thread.daemon = True  # 设置为守护线程，这样当Jupyter关闭时线程会自动结束
server_thread.start()

# 等待服务器启动
time.sleep(3)  # 增加等待时间，确保服务器完全启动


INFO:     Started server process [104947]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


In [6]:
try:
    # 调用API示例
    url = "http://localhost:8000/generate"
    headers = {"Content-Type": "application/json"}
    data = {"text": "讲解一下LangChain的主要功能"}
    # 
    response = requests.post(url, headers=headers, data=json.dumps(data))
    print("API响应结果:")
    display(Markdown(response.json()['response']['content']))
except Exception as e:
    print(f"请求发生错误: {str(e)}")

2025-03-25 12:19:06,211 - __main__ - INFO - 收到请求: 讲解一下LangChain的主要功能
2025-03-25 12:19:06,355 - httpx - INFO - HTTP Request: POST https://api.deepseek.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-03-25 12:19:27,849 - __main__ - INFO - 请求处理成功


INFO:     127.0.0.1:54756 - "POST /generate HTTP/1.1" 200 OK


API响应结果:


**LangChain的主要功能**  

**1. 模块化组件（Modular Components）**  
提供预构建的模块（如模型调用、记忆管理、数据检索等），支持灵活组合以构建复杂应用。  

**2. 模型集成（Model Integration）**  
支持多种大语言模型（如OpenAI、Anthropic、Hugging Face等），统一接口简化调用流程。  

**3. 数据增强检索（Retrieval-Augmented Generation, RAG）**  
结合外部数据源（如文档、数据库），通过检索增强生成内容的准确性和相关性。  

**4. 记忆管理（Memory）**  
支持短期（会话级）和长期（数据库）记忆存储，维持多轮对话的上下文连贯性。  

**5. 代理与工具链（Agents & Tools）**  
允许模型通过工具（如搜索、计算）与外部系统交互，实现动态任务执行（如自动网页搜索、API调用）。  

**6. 链式工作流（Chains）**  
将多个模块串联成自动化流程（如“提问→检索→生成”），适用于复杂任务编排。  

**7. 评估与调试（Evaluation）**  
提供工具评估模型输出效果，帮助优化提示（prompt）和流程设计。  

---  

**Main Features of LangChain**  

**1. Modular Components**  
Pre-built modules (e.g., model calls, memory, data retrieval) enable flexible assembly of complex applications.  

**2. Model Integration**  
Unified interface for multiple LLMs (OpenAI, Anthropic, Hugging Face, etc.), simplifying model calls.  

**3. Retrieval-Augmented Generation (RAG)**  
Enhances responses by integrating external data (documents, databases) into generation.  

**4. Memory Management**  
Supports short-term (conversation) and long-term (database) memory to maintain context.  

**5. Agents & Tools**  
Models interact with external systems via tools (e.g., search, APIs) for dynamic task execution.  

**6. Chains**  
Automates multi-step workflows (e.g., "question→retrieve→generate") for complex tasks.  

**7. Evaluation**  
Tools to assess output quality and optimize prompts/pipelines.

## 2. 监控与维护

### 2.1 性能监控

In [7]:
from langchain.callbacks import get_openai_callback

def monitor_performance():
    with get_openai_callback() as cb:
        response = chain.run(input)
        print(f"Token使用量: {cb.total_tokens}")
        print(f"API调用次数: {cb.successful_requests}")

In [13]:
from langchain_deepseek import ChatDeepSeek

# 演示性能监控
input_text = "解释一下向量数据库的作用"
with get_openai_callback() as cb:
    chain = ChatDeepSeek(model='deepseek-chat')
    response = chain.invoke(input_text)
    print(f"查询: {input_text}")
    print(f"Token使用量: {cb.total_tokens}")
    print(f"API调用次数: {cb.successful_requests}")
    print(f"总花费: ${cb.total_cost}")

2025-03-25 12:30:07,402 - httpx - INFO - HTTP Request: POST https://api.deepseek.com/v1/chat/completions "HTTP/1.1 200 OK"


查询: 解释一下向量数据库的作用
Token使用量: 756
API调用次数: 1
总花费: $0.0


In [14]:
Markdown(response.content)

向量数据库（Vector Database）是一种专门用于存储、检索和管理**向量数据（高维数值数组）**的数据库系统。它的核心作用是高效处理基于向量相似度的搜索和分析，广泛应用于机器学习、人工智能和大数据领域。以下是其核心作用及关键特点的详细解释：

---

### **1. 核心作用**
#### （1）**相似性搜索**
   - **问题场景**：给定一个查询向量（如图片、文本的嵌入表示），快速找到数据库中与之最相似的向量。
   - **解决方案**：通过计算向量间的距离（如余弦相似度、欧氏距离），返回最接近的结果。
   - **典型应用**：
     - 图片/视频检索（如以图搜图）。
     - 推荐系统（相似商品、用户兴趣匹配）。
     - 自然语言处理（语义搜索、问答系统）。

#### （2）**高效存储与索引**
   - 传统数据库无法高效处理高维向量，而向量数据库使用**近似最近邻（ANN）算法**（如HNSW、IVF、LSH）加速搜索，在精度和速度间取得平衡。
   - 支持海量向量数据的持久化存储和实时更新。

#### （3）**多模态数据处理**
   - 可将不同模态（文本、图像、音频）转换为统一向量空间，实现跨模态搜索（如用文本搜索图片）。

#### （4）**AI模型集成**
   - 作为AI应用的底层基础设施，存储模型生成的嵌入向量（Embeddings），支撑模型推理和微调。

---

### **2. 关键特点**
- **高维索引**：优化数十到数千维向量的检索效率。
- **实时性**：支持低延迟查询，适合在线服务（如聊天机器人）。
- **可扩展性**：分布式架构支持水平扩展，应对亿级数据。
- **混合查询**：结合向量搜索与传统结构化查询（如过滤标签+相似性搜索）。

---

### **3. 与传统数据库对比**
| **能力**               | **传统数据库**       | **向量数据库**               |
|------------------------|---------------------|-----------------------------|
| 数据类型               | 结构化数据（表格）   | 高维向量+结构化元数据         |
| 查询方式               | 精确匹配（SQL）      | 相似性搜索（距离计算）        |
| 索引优化               | B树、哈希索引        | ANN算法（HNSW、IVF等）       |
| 典型场景               | 交易记录、用户管理   | 推荐系统、语义搜索            |

---

### **4. 典型应用场景**
- **推荐系统**：根据用户历史行为向量推荐相似内容。
- **AIGC**：存储生成式AI的嵌入，实现个性化生成（如Diffusion模型检索）。
- **异常检测**：通过向量距离识别异常数据（如金融欺诈）。
- **生物信息学**：基因序列相似性分析。

---

### **5. 常见工具**
- **专用向量数据库**：Milvus、Pinecone、Weaviate、Qdrant。
- **扩展向量功能的数据库**：PostgreSQL（pgvector）、Redis（RedisSearch）。
- **云服务**：Google Vertex AI Matching Engine、AWS Aurora ML。

---

### **总结**
向量数据库通过优化高维向量的存储和检索，解决了AI时代非结构化数据（如图片、文本）的相似性搜索难题，成为构建智能应用的关键组件。其核心价值在于**将复杂的语义匹配转化为高效的数学计算**，从而释放数据的深层关联性。